In [8]:
import os
import pandas as pd
from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer


In [9]:
# Create processed folder if not exists
os.makedirs("data/processed", exist_ok=True)

# Initialize VADER
analyzer = SentimentIntensityAnalyzer()


In [10]:
# Function to compute sentiment scores
def compute_sentiment(text):
    scores = analyzer.polarity_scores(str(text))
    return pd.Series([scores['neg'], scores['neu'], scores['pos'], scores['compound']])


In [30]:
# Define file paths for each bank with exact file names
banks = {
    "dashen": "notebooks/data/Dashen Bank_reviews_cleaned.csv",
    "cbe": "notebooks/data/CBE_reviews_cleaned.csv",
    "boa": "notebooks/data/BOA_reviews_cleaned.csv"
}


In [31]:
# Apply sentiment analysis to each file and save result
for bank, filepath in banks.items():
    try:
        df = pd.read_csv(filepath)
        # Check if 'review' column exists, else raise meaningful error
        if 'review' not in df.columns:
            raise KeyError(f"'review' column not found in {filepath}")
        
        df[['neg', 'neu', 'pos', 'compound']] = df['review'].apply(compute_sentiment)
        df['sentiment'] = df['compound'].apply(
            lambda score: 'positive' if score >= 0.05 else 'negative' if score <= -0.05 else 'neutral'
        )
        output_file = f"data/processed/{bank}_reviews_sentiment.csv"
        df.to_csv(output_file, index=False)
        print(f"[✔] Saved sentiment analysis for {bank.title()} to {output_file}")
    except Exception as e:
        print(f"[✘] Failed to process {bank.title()}: {e}")


[✔] Saved sentiment analysis for Dashen to data/processed/dashen_reviews_sentiment.csv
[✔] Saved sentiment analysis for Cbe to data/processed/cbe_reviews_sentiment.csv
[✔] Saved sentiment analysis for Boa to data/processed/boa_reviews_sentiment.csv
